In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

batch_size = 64
lr = 0.001
epochs = 20

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Using device: cpu


In [12]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32 , kernel_size = 3,padding = 0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,14, kernel_size = 3,padding = 0)
        self.bn2 = nn.BatchNorm2d(14)
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.fc1 = nn.Linear(14*5*5,10)
        self.dropout = nn.Dropout(0.1)
    def forward(self,x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = torch.flatten(x,1)

        x = self.dropout(x)       
        x = self.fc1(x)
        return x

model = CNN().to(device)
print(model)

def count_params(layer):
    return sum(p.numel() for p in layer.parameters())

print("Conv1:", count_params(model.conv1))
print("Conv2:", count_params(model.conv2))
print("FC1:", count_params(model.fc1))

total = sum(p.numel() for p in model.parameters())
print("Total Parameters:", total)


CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 14, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=350, out_features=10, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
Conv1: 320
Conv2: 4046
FC1: 3510
Total Parameters: 7968


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = lr)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for image,labels in train_loader:
        image,labels = image.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

In [14]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 99.15%
